# Install Detectron2 Dependencies

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

In [ ]:
# install detectron2:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog

# Mount To My Drive

In [ ]:
#mount to my drive
from google.colab import drive
drive.mount('/content/drive')

# Import and Register Custom Detectron2 Data

In [ ]:
# !curl -L "/content/drive/MyDrive/" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
!unzip /content/drive/MyDrive/infor_card_json.zip > /dev/null


In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "/content/coco_json/train/_annotations.coco.json", "/content/coco_json/train")
register_coco_instances("my_dataset_val", {}, "/content/coco_json/valid/_annotations.coco.json", "/content/coco_json/valid")
register_coco_instances("my_dataset_test", {}, "/content/coco_json/test/_annotations.coco.json", "/content/coco_json/test")


In [ ]:
#visualize training data
my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train")
dataset_dicts = DatasetCatalog.get("my_dataset_train")

import random
from detectron2.utils.visualizer import Visualizer

for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=my_dataset_train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

# Train Custom Detectron2 Detector

In [ ]:
#We are importing our own Trainer Module here to use the COCO validation evaluation during training. Otherwise no validation eval occurs.

from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [ ]:
# import torch
# torch.cuda.empty_cache()
# torch.cuda.memory_summary(device=None, abbreviated=False)

In [ ]:
from detectron2.config import get_cfg
import os

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.OUTPUT_DIR=("/content/drive/MyDrive/ID_CARD/Detect_infor/model_coco_X101")
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)

cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 1
cfg.SOLVER.BASE_LR = 0.00025


# cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 1500 #adjust up if val mAP is still rising, adjust down if overfit
cfg.SOLVER.STEPS = (1000, 1500)
cfg.SOLVER.GAMMA = 0.05


cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 32
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 9 #your number of classes + 1

# cfg.TEST.EVAL_PERIOD = 500


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg)
# trainer.resume_or_load(resume=False)
# trainer.train()

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
#evaluate my model

#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.config import get_cfg
#from detectron2.evaluation.coco_evaluation import COCOEvaluator
import os

# cfg = get_cfg()
# cfg.OUTPUT_DIR=("/content/drive/MyDrive/ID_CARD/Detect_infor/model_coco_X101")
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "my_dataset_test")
inference_on_dataset(trainer.model, val_loader, evaluator)

# Download VietOCR

In [ ]:
! pip install --quiet vietocr==0.3.2

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg

In [ ]:
config = Cfg.load_config_from_name('vgg_transformer')

In [ ]:
# config['weights'] = './weights/transformerocr.pth'
config['weights'] = 'https://drive.google.com/uc?id=13327Y1tz1ohsm5YZMyXVMPIOjoOA0OaA'
config['cnn']['pretrained']=False
config['device'] = 'cuda:0'
config['predictor']['beamsearch']=False

In [ ]:
detector = Predictor(config)

# Inference with Detectron2 Saved Weights and VietOCR



In [ ]:
#load model weights
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/content/drive/MyDrive/ID_CARD/Detect_infor/model_coco_X101/model_final.pth")
cfg.DATASETS.TEST = ("my_dataset_test", )
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
predictor = DefaultPredictor(cfg)
test_metadata = MetadataCatalog.get("my_dataset_test")

In [ ]:
from detectron2.utils.visualizer import ColorMode
import glob
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import cv2

In [ ]:
from PIL import Image
import numpy 
import string

def inference_OCR(img):
  cv2_imshow(img)
  image=Image.fromarray(numpy.uint8(img))
  s=detector.predict(image)
  print(s)


def cropped_images(imageName):
  im = cv2.imread(imageName)
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],
                metadata=test_metadata, 
                scale=0.8
                 )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])

  #shape of original_image
  original_h, original_w = im.shape[:2]

  #extract 4-points and categories ID
  pred_box=outputs["instances"].get('pred_boxes').tensor
  pred_class=outputs["instances"].get('pred_classes')

  box=[]
  for i in range(len(pred_box)):
    temp=[]
    for j in range(0,4):
      temp.append(int(pred_box[i][j].item()))
    box.append(temp)

  #extract identify number classes
  class_id=[]
  for i in range(len(pred_class)):
    class_id.append(int(pred_class[i].item()))

  #sorted identify number classes
  list_id=[]
  count_1=0
  count_5=0
  for index,element in enumerate(class_id):
    list_id.append([element,index])
    if element==1: count_1+=1
    if element==5: count_5+=1
  list_id= sorted(list_id[:])

  #append sub-image from image id-card
  list_cropped=[]
  for i in range(len(class_id)):
      [xmin, ymin, xmax, ymax]= box[list_id[i][1]]
      cropped = im[int(ymin):int(ymax),int(xmin):int(xmax)]
      list_cropped.append(cropped)

  #show out sub-image results
  print("______________________________________________________________________________________________\n")

  if count_1==2: pred_1=1
  else: pred_1=0
  if count_5==2: pred_5=1
  else: pred_5=0

  print("Identify number: ")
  inference_OCR(list_cropped[7+pred_1+pred_5])

  print('-----------------------------------------------')
  print("Name: ")
  inference_OCR(list_cropped[5+pred_1+pred_5])

  print('-----------------------------------------------')
  print("Date of Birth: ")
  inference_OCR(list_cropped[1+pred_1])

  print('-----------------------------------------------')
  print("Gender: ")
  inference_OCR(list_cropped[3+pred_1])

  print('-----------------------------------------------')
  print("Nationality: ")
  inference_OCR(list_cropped[6+pred_1+pred_5])

  print('-----------------------------------------------')
  print("Native land: ")
  if count_1==2:
    # im=cv2.hconcat([list_cropped[0+pred_1],list_cropped[0]])
    inference_OCR(list_cropped[0+pred_1])
    inference_OCR(list_cropped[0])
 
  else:
    inference_OCR(list_cropped[0])

  print('-----------------------------------------------')
  print("Permanent address: ")
  if count_5==2:
    # im=cv2.hconcat([list_cropped[4+pred_1+pred_5]],list_cropped[4+pred_1])
    inference_OCR(list_cropped[4+pred_1+pred_5])
    inference_OCR(list_cropped[4+pred_1])
  else:
    inference_OCR(list_cropped[4+pred_1])
 
  print('-----------------------------------------------')
  print("Expiry date: ")
  inference_OCR(list_cropped[2+pred_1])


In [ ]:
!pip install opencv-contrib-python

In [ ]:
from detectron2.utils.visualizer import ColorMode
import glob
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import cv2
import os

for imageName in glob.glob('/content/drive/MyDrive/ID_CARD/images_infor/*jpg'):
  cropped_images(imageName)